In [5]:
import gurobipy as gp
from gurobipy import GRB
import itertools as it

# Disable stdout output from gurobipy
env = gp.Env(empty=True)
env.setParam("OutputFlag", 0)
env.start()

None

In [6]:
# Input parsing

Obligation = tuple[int, int, int]
Student = list[Obligation]

file = open("inputs/input3.txt")

t = int(file.readline())
m = int(file.readline())
b = [int(bh) for bh in file.readline().split(",")]
n = int(file.readline())

students: list[Student] = []
for i in range(n):
    o, *line = [int(x) for x in file.readline().split(",")]
    obligations = []
    for x in range(o):
        lb, ub, duration = line[x*3:(x+1)*3]
        obligations.append((lb - 1, ub - 1, duration))
    students.append(obligations)

## Obligation planning

We plan the students' obligations optimally around a given configuration of borrels such that the obligations needed to be done during borrels is minimised. For this we have constructed the following ILP:

**Decision variables**:
$s_{ijt}\in\{0,1\}$: student $i$ does work for obligation $j$ at time $t$

**Objective**: We minimise the total time spent working on obligations by all students during the planned borrels. The objective function does not represent the number of points the borrel configuration has in the way as it is calculated in the algorithm's final output. This is done as a separate post-processing step after the borrel configuration with the highest objective has been found.
$$
z=\min\sum_{h=1}^m\sum_{i=1}^n\sum_{j=1}^{\ell_i}\sum_{t=c_h}^{c_h+b_h}s_{ijt}
$$

**Constraints**:

1. The total time spent on an obligation should equal the required time
$$
\sum_{k=r_j}^{d_j}s_{ijk}=p_{ij}\quad\forall i\leq n, j\leq\ell_i
$$
2. A student cannot work on more than one obligation at the same time
$$
\sum_{j=1}^{\ell_i}s_{ijk}\leq1\quad\forall i\leq n, k\leq t
$$

In [7]:
class BorrelILP:
    def __init__(self, starts):
        self.borrels = starts
        self.model = gp.Model(env = env)

        s_tuples = [
            (i, j, k)
            for i, student in enumerate(students)
            for j, _ in enumerate(student)
            for k in range(t)
        ] 
        self.s = self.model.addVars(
            s_tuples,
            name=[f"s_{i},{j},{k}" for i, j, k in s_tuples],
            vtype=GRB.BINARY
        )

        for s_index, student in enumerate(students):
            for o_index, (start, end, duration) in enumerate(student):
                self.model.addConstr(gp.quicksum(self.s[s_index, o_index, k] for k in range(start, end + 1)) == duration)

            for time in range(t):
                self.model.addConstr(self.s.sum(s_index, "*", time) <= 1)

        self.model.setObjective(
            gp.quicksum(
                self.s[i, j, t]
                for h in range(m)
                for i in range(n)
                for j in range(len(students[i]))
                for t in range(starts[h], starts[h] + b[h])
            ),
            GRB.MINIMIZE
        )
    
    def optimise(self):
        self.model.optimize()
        return self.s

## Finding the best borrel configuration

To find the optimal borrel configuration, we simply use a brute-force strategy. We create a list of all possible borrel starting times and for each one construct an ILP and solve it. As described earlier, we manually need to calculate the points according to the problem specification. If we found a configuration with a higher score we save the model and the score, otherwise we discard the model. After having tried all configuration we print the required information to the console.  

In [8]:
highest_model = None
highest_points = 0

for x in it.product(*[range(t - length + 1) for length in b]):
    model = BorrelILP(list(x))
    model.optimise()

    borrel_attendance = []
    points = 0
    for h, start in enumerate(list(x)):
        for i, student in enumerate(students):
            all_empty = all(
                model.s[i, j, k].x == 0
                and (i, k) not in borrel_attendance
                for k in range(start, start + b[h])
                for j in range(len(student))
            )
            if all_empty:
                for k in range(start, start + b[h]):
                    borrel_attendance.append((i, k))
                points += int(all_empty)

    if points > highest_points:
        highest_points = points
        highest_model = model

# Printing output

print(", ".join(str(borrel + 1) for borrel in highest_model.borrels))
for i, student in enumerate(students):
    for j, obligation in enumerate(student):
        times = [str(k + 1) for k in range(t) if highest_model.s[i, j, k].x == 1]
        print(", ".join(times))
print(highest_points)

1, 11, 4
2, 3
7, 8, 9, 10
8, 9, 10
2, 3, 4
5, 6, 7
8, 9
2, 3
7, 8, 9
10
11
